In [19]:
import torch
from torch import nn
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, json, string, os
import collections

from argparse import Namespace
from IPython.display import display 
from collections import Counter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
     

In [128]:
df = pd.read_csv('yelp_review.csv')

In [3]:
df.head()

,Unnamed: 0,text,stars
0,0,"If you decide to eat here, just be aware it is...",3.0
1,1,I've taken a lot of spin classes over the year...,5.0
2,2,Family diner. Had the buffet. Eclectic assortm...,3.0
3,3,"Wow! Yummy, different, delicious. Our favo...",5.0
4,4,Cute interior and owner (?) gave us tour of up...,4.0


In [129]:
df = df[['text','stars']]
df['stars'] = df['stars'].apply(lambda x: 1 if x>=3 else 0)

In [130]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['stars'],test_size=0.2,stratify = df['stars'])

In [131]:
train = pd.DataFrame({'text':X_train,'rating':y_train})
test = pd.DataFrame({'text':X_test,'rating':y_test})

In [132]:
train = train.reset_index().drop('index',axis=1)
test = test.reset_index().drop('index',axis=1)

In [133]:
train.head()
train.shape

(80000, 2)

In [134]:
test.head()

,text,rating
0,Perfectly suitable property. The location is a...,1
1,The food is amazing especially the Hummus. The...,1
2,I stumbled in here on accident and hoped I had...,0
3,Ryan Christopher's (formerly Caviston's) is si...,1
4,"Horrible service center staff, it took 4 hours...",0


In [135]:
import collections

In [136]:
train.rating.value_counts()

1    64873
0    15127
Name: rating, dtype: int64

In [155]:
by_rating = collections.defaultdict(list)
#creating list of dict with each row
for index, row in train.iterrows():
    #grouping on rating cat in a dic form 
    by_rating[row.rating].append(row.to_dict()) #{1:[{text:,rating:}],0:[{text:,rating:}]},]
final_list = []

np.random.seed(1337)

for key,item_list in sorted(by_rating.items()): #{key:[{values}],key:[{values}]}
    np.random.shuffle(item_list)
    
    n_total = len(item_list)
    n_train = int(0.7*n_total)
    n_val =   int(0.15*n_total)
    n_test =   int(0.15*n_total)
    
    for item in item_list[:n_train]:
        item["split"] = "train"
    for item in item_list[n_train:n_train+n_val]:
        item["split"] = "val"
    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item["split"] = "test" 
  #@ Adding to the Final List:
    final_list.extend(item_list)

#@ Creating the Final DataFrame:
final_reviews = pd.DataFrame(final_list)
     

In [165]:
final_reviews = final_reviews.dropna()

In [166]:
final_reviews['split'].value_counts()

train    55999
test     11999
val      11999
Name: split, dtype: int64

In [167]:
final_reviews.to_csv('final_yelp.csv', index=False)

In [81]:
a = {1:[{'a':10,'b':22}],2:[{'a':10,'b':22}]}
a.items()

dict_items([(1, [{'a': 10, 'b': 22}]), (2, [{'a': 10, 'b': 22}])])

In [138]:
print(len(by_rating))
print(len(by_rating[1]) ,len(by_rating[0]))
print(by_rating.keys())

2
64873 15127
dict_keys([1, 0])


In [151]:
final_list

[{'text': "Downingtown don't even bother. Only 5 4k 49'. Not impressed.  Very disappointed. Guy came out have out wristbands. By the time he reached the 20tj person no more. I ask how many they had he stated 5.",
  'rating': 0,
  'split': 'train'},
 {'text': 'Cocktails are fine but service is pretentious and condescending.  \n\nThe bar was fairly crowded the night we went, and the one bartender and one waitress were clearly overwhelmed.  We were condescendingly told by the bartender that because he was an older individual (older than us), that he could not handle a large group of people at his bar and that he would not provide the level of service he typically provides (the implication is that typical patrons are more worthy of service than us).  Waitress and bartender also had a fixation on everyone sitting down and would not allow us to stand up to mingle between tables. \n\nWould be a good place if you are alittle bit older, like pretension, worn down furniture, and a nice, formal c

In [168]:
import re
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text) #adds space before and after a punct
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text
final_reviews.text = final_reviews.text.apply(preprocess_text)

C:\Users\Chinmay\anaconda3\envs\envpytorch\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [64]:
review = ['good','bad','okay']
ratings = [1,0,1.5]
dataset = ReviewDataset.from_vocabularies(review,ratings)

['good', 'bad', 'okay', 'asc'] [1, 0, 1.5, 0]


In [169]:
set(final_reviews['rating'])

{0, 1}

# Pytorch Dataset Class

In [170]:


#@ PyTorch Dataset Class:
class ReviewDataset(Dataset):
    
    def __init__(self, review_df, vectorizer):
        
        """
        Args: review_df(pandas.DataFrame): The dataset.
            : vectorizer(ReviewVectorizer): Vector instantiated from dataset.
        """
        self.review_df = review_df
        self._vectorizer = vectorizer

        self.train_df = self.review_df[self.review_df.split == "train"]
        self.train_size = len(self.train_df)

        self.val_df = self.review_df[self.review_df.split == "val"]
        self.validation_size = len(self.val_df)

        self.test_df = self.review_df[self.review_df.split == "test"]
        self.test_size = len(self.test_df)

        self._lookup_dict = {"train": (self.train_df, self.train_size),
                           "val": (self.val_df, self.validation_size),
                           "test": (self.test_df, self.test_size)}
        self.set_split("train")
  
    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """Load dataset and make new vectorizer from scratch.
        Args: review_csv: Location of the dataset.
        Returns: An instance of ReviewDataset.
        """
        review_df = pd.read_csv(review_csv)
        train_review_df = review_df[review_df.split == "train"]
        return cls(review_df, ReviewVectorizer.from_dataframe(train_review_df))
  
    @classmethod
    def load_dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer.
        Args: review_csv: Location of the dataset.
        : vectorizer_filepath: Location of the saved vectorizer.
        Returns: An instance of the ReviewDataset.
        """
        review_df = pd.read_csv(review_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(review_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """A static method for loading the vectorizer from file.
        Args: vectorizer_filepath: Location of serialized vectorizer.
        Returns: An instance of ReviewVectorizer.
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)
  
    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)
  
    def get_vectorizer(self):
        """Returns the Vectorizer.
        """
        return self._vectorizer
  
    def set_split(self, split="train"):
        """Splits the dataset using a column in the DataFrame.
        Args: split(str): One of "train", "val" or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size
  
    def __getitem__(self, index):
        """Primary entry point of PyTorch Datasets.
        Args: index: Index of the Datapoint.
        Returns: A dictionary holding the Data point features and labels.
        """
        row = self._target_df.iloc[index]
        review_vector = self._vectorizer.vectorize(row.text)
        rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)
        return {"x_data": review_vector,
                "y_target": rating_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the Dataset.
        Args: batch_size(int)
        Returns: Number of batches in the Dataset.
        """
        return len(self) // batch_size

# The Vocabulary Class

In [220]:
class Vocabulary(object):
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
        token_to_idx (dict): a pre­existing map of tokens to indices
        add_unk (bool): a flag that indicates whether to add the UNK token
        unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token  for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)

    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    
    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    def add_token(self, token):
        """Update mapping dicts based on the token.
        Args:
        token (str): the item to add into the Vocabulary
        Returns:
        index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
        token (str): the token to look up
        Returns:
        index (int): the index corresponding to the token
        Notes:
        `unk_index` needs to be >=0 (having been added into the Vocabulary)
        for the UNK functionality
        """
        if self.add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
        index (int): the index to look up
        Returns:
        token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    
    def __len__(self):
        return len(self._token_to_idx)

# Vectorizer Class

In [235]:
class ReviewVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, review_vocab, rating_vocab):
        """
        Args:
        review_vocab (Vocabulary): maps words to integers
        rating_vocab (Vocabulary): maps class labels to integers
        """
        self.review_vocab = review_vocab
        self.rating_vocab = rating_vocab
 
    def vectorize(self, review):
        """Create a collapsed one­hit vector for the review
        Args:
        review (str): the review
        Returns:
        one_hot (np.ndarray): the collapsed one­hot encoding
        """
        one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
        for token in review.split(" "):
            if token not in string.punctuation:
                one_hot[self.review_vocab.lookup_token(token)] = 1
        return one_hot
    
    @classmethod
    def from_dataframe(cls, review_df, cutoff=25):
        review_vocab = Vocabulary(add_unk=True)
        rating_vocab = Vocabulary(add_unk=False)
        for rating in sorted(set(review_df.rating)):
            rating_vocab.add_token(rating)
       
        word_counts = Counter()
        for review in review_df.text:
            for word in review.split(" "):
                if word not in string.punctuation:
                    word_counts[word] += 1
                    
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)
        return cls(review_vocab, rating_vocab)
    
    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a ReviewVectorizer from a serializable dictionary
        Args:
        contents (dict): the serializable dictionary
        Returns:
        an instance of the ReviewVectorizer class
        """
        review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
        rating_vocab = Vocabulary.from_serializable(contents['rating_vocab'])
        return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)
    
    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
        contents (dict): the serializable dictionary
        """
        return {'review_vocab': self.review_vocab.to_serializable(),
            'rating_vocab': self.rating_vocab.to_serializable()}

# DataLoader

In [222]:
#@ DataLoader:
def generate_batches(dataset, batch_size, shuffle=True, 
                         drop_last=True, device="gpu"):
    """A generator function which wraps the PyTorch DataLoader. 
     """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, 
                          shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            print(name,tensor)
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [202]:
final_reviews[final_reviews['split']=='train'][:10]

,rating,split,text
0,0,train,downingtown don t even bother . only k . not i...
1,0,train,cocktails are fine but service is pretentious ...
2,0,train,steaks ordered were undercooked . just not co...
3,0,train,very disappointed in them . i made an appointm...
4,0,train,bought lancer evo . saturday at the end of nov...
5,0,train,my husband and i are regular customers here . ...
6,0,train,"not a great vibe , employees were pretty rude ..."
7,0,train,please use your room safe ! ! ! i m very torn ...
8,0,train,apparently they make a point to come tell me t...
9,0,train,we didn t really have a good experience at thi...


In [212]:
dataset.set_split("train")

In [216]:
batch_generator = generate_batches(dataset,
                                       batch_size=args.batch_size,
                                       device=args.device)

for batch_index, batch_dict in enumerate(batch_generator):
    print(batch_dict)

AttributeError: 'Vocabulary' object has no attribute 'add_unk'

# Perceptron

In [174]:
#@ A Perceptron Classifier:
class ReviewClassifier(nn.Module):
    """A simple Perceptron based Classifier.
    """
    def __init__(self, num_features):
        """
       Args: num_features(init): The size of input feature vector.
        """
        super(ReviewClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features,out_features=1)

    def forward(self, x_in, apply_sigmoid=False):
        """Forward pass of the Classifier.
        Args: x_in(Tensor): An input data Tensor.
        : apply_sigmoid(bool): A flag for the sigmoid activation.
        """
        y_out = self.fc1(x_in).squeeze()
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

In [75]:
#@ Hyperparameters and Program Options:
args = Namespace(
    #@ Data and path information:
    frequency_cutoff=25,
    model_state_file="model.pth",
    review_csv="final_yelp.csv",
    save_dir="model_storage",
    vectorizer_file="vectorizer.json",
    batch_size=128,
    early_stopping_criteria=5, 
    learning_rate=0.001,
    num_epochs=100,
    seed=42,
    #@ Runtime Options:
    catch_keyword_interrupt=True,
    cuda=True,
    expand_filepaths_to_sava_dir=True,
    reload_from_files=False
)

if args.expand_filepaths_to_sava_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)
    print("Expanded Filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

#@ Checking the CUDA:
if not torch.cuda.is_available():
    args.cuda=False
print("Using CUDA: {}".format(args.cuda))
args.device = torch.device("cuda" if args.cuda else "cpu")

#@ Seed for Reproducibility:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)
set_seed_everywhere(args.seed, args.cuda)

#@ Handle Dirs:
def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
handle_dirs(args.save_dir)

Expanded Filepaths: 
	model_storage\vectorizer.json
	model_storage\model.pth
Using CUDA: True


In [223]:
dataset = ReviewDataset.load_dataset_and_make_vectorizer('final_yelp.csv')
#In load_and_make ,df is read, split by train col . return tow class obj, one for review_df and one is 
#RevVect class object calling from_dataframe
#in from_databse in REV_VECT train_df is passed. here tow Vocabulary class obj is called
#review_vocan and rating_vocab and all param in Vocab class gets initialized.
#this gets initlaized as get_vector in Rev_Dataclass
vectorizer = dataset.get_vectorizer()

asd
init for review_vocab
init for rating_vocab
rating is 0
rating is 1
<Vocabulary(size=10653)> <Vocabulary(size=2)>

printing updated review_vocab <Vocabulary(size=10653)>
printing updated rating_vocab <Vocabulary(size=2)>


In [93]:
dataset.review_df.head()

,rating,split,text
0,0,train,after all the great reviews i thought this pla...
1,0,train,"the service was quick , and the owner was frie..."
2,0,train,cru is a wonderful place wether youre just gra...
3,0,train,average food . they don t even know the differ...
4,0,train,"we had several quotes , with pete and ron s be..."


In [121]:
set(dataset._lookup_dict['train'][0]['rating'])

{0}

In [102]:
len(vectorizer.review_vocab)

7388

In [217]:
def make_train_state(args):
    return {
      "stop_early": False,
      "early_stopping_step":0,
      "early_stopping_best_val":1e8,
      "learning_rate":args.learning_rate,
      "epoch_index":0,
      "train_loss":[],
      "train_acc":[],
      "val_loss":[],
      "val_acc":[],
      "test_loss":-1,
      "test_acc":-1,
      "model_filename":args.model_state_file
     }
def update_train_state(args, model, train_state):
    #@ Saving atleast One Model:
    if train_state["epoch_index"] == 0:
        torch.save(model.state_dict(), train_state["model_filename"])
        train_state["stop_early"] = False
    #@ Saving the Model if performance improved:
    elif train_state["epoch_index"] >= 1:
        loss_tm1, loss_t = train_state["val_loss"][-2:]
    #@ If Loss is worsened:
        if loss_t >= train_state["early_stopping_best_val"]:
            train_state["early_stopping_step"] += 1
        else:
            if loss_t < train_state["early_stopping_best_val"]:
                torch.save(model.state_dict(), train_state["model_filename"])
            train_state["early_stopping_step"] = 0
        train_state["stop_early"] = train_state["early_stopping_step"] >= args.early_stopping_criteria
    return train_state

def compute_accuracy(y_pred, y_target):
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).long()
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [236]:
#@ Initializing the Data Training:
if args.reload_from_files:
    print("Loading the Dataset and Vectorizer")
    dataset = ReviewDataset.load_dataset_and_load_vectorizer(args.review_csv, args.vectorizer_file)
else:
    print("Loading the Dataset and Creating the Vectorizer")
    dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv)
    dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()
classifier = ReviewClassifier(num_features=len(vectorizer.review_vocab))

Loading the Dataset and Creating the Vectorizer


In [237]:
classifier = classifier.to(args.device)
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
schedular = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode="min", factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm_notebook(
    desc="training routine",
    total=args.num_epochs,
    position=0
)

dataset.set_split("train")
train_bar = tqdm_notebook(
    desc="split=train",
    total=dataset.get_num_batches(args.batch_size),
    position=1,
    leave=True
)

dataset.set_split("val")
val_bar = tqdm_notebook(
    desc="split=val",
    total=dataset.get_num_batches(args.batch_size),
    position=1,
    leave=True
)

try:
    for epoch_index in range(args.num_epochs):
        train_state["epoch_index"] = epoch_index
        #@ Iterating over Training Dataset:
        dataset.set_split("train")
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            #@ Step1: Zero Gradients.
            optimizer.zero_grad()
            #@ Step2: Computing the Output:
            y_pred = classifier(x_in=batch_dict["x_data"].float())
            #@ Step3: Computing the Loss:
            loss = loss_func(y_pred, batch_dict["y_target"].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            #@ Step4: Using loss to produce gradients:
            loss.backward()
            #@ Step5: Using optimizer to take gradient step:
            optimizer.step()
            #@ Computing the accuracy:
            acc_t = compute_accuracy(y_pred, batch_dict["y_target"])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            #@ Updating:
            train_bar.set_postfix(loss=running_loss,
                                acc=running_acc,
                                epoch=epoch_index)
            train_bar.update()
        train_state["train_loss"].append(running_loss)
        train_state["train_acc"].append(running_acc)

        #@ Iterating over Validation Dataset:
        dataset.set_split("val")
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()
        for batch_index, batch_dict in enumerate(batch_generator):
          #@ Computing the Ouput:
            y_pred = classifier(x_in=batch_dict["x_data"].float())
          #@ Computing the Loss:
            loss = loss_func(y_pred, batch_dict["y_target"].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
          #@ Computing the accuracy:
            acc_t = compute_accuracy(y_pred, batch_dict["y_target"])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
          #@ Updating:
            val_bar.set_postfix(loss=running_loss,
                              acc=running_acc,
                              epoch=epoch_index)
            val_bar.update()
        train_state["val_loss"].append(running_loss)
        train_state["val_acc"].append(running_acc)

        train_state = update_train_state(args=args, model=classifier, 
                                         train_state=train_state)
        schedular.step(train_state["val_loss"][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

        if train_state["stop_early"]:
            break
        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting Loop")


C:\Users\Chinmay\anaconda3\envs\envpytorch\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


training routine:   0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Chinmay\anaconda3\envs\envpytorch\lib\site-packages\ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


split=train:   0%|          | 0/437 [00:00<?, ?it/s]

C:\Users\Chinmay\anaconda3\envs\envpytorch\lib\site-packages\ipykernel_launcher.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


split=val:   0%|          | 0/93 [00:00<?, ?it/s]

AttributeError: 'Vocabulary' object has no attribute 'add_unk'

In [ ]:
#@ Computing the Loss and Accuracy on the Test Dataset:
classifier.load_state_dict(torch.load(train_state["model_filename"]))
classifier = classifier.to(args.device)
dataset.set_split("test")
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
  #@ Computing the Output:
    y_pred = classifier(x_in = batch_dict["x_data"].float())
  #@ Computing the Loss:
    loss = loss_func(y_pred, batch_dict["y_target"].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)
  #@ Computing the Accuracy:
    acc_t = compute_accuracy(y_pred, batch_dict["y_target"])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state["test_loss"] = running_loss
train_state["test_acc"] = running_acc

print("Test loss: {:.3f}".format(train_state["test_loss"]))
print("Test accuracy: {:.3f}".format(train_state["test_acc"]))

In [83]:


#@ Predicting the Rating:
def predict_rating(review, classifier, vectorizer, decision_threshold=0.5):
    review = preprocess_text(review)
    vectorized_review = torch.tensor(vectorizer.vectorize(review))
    result = classifier(vectorized_review.view(1, -1))
    probability_value = F.sigmoid(result).item()
    index=1
    if probability_value < decision_threshold:
        index=0
    return vectorizer.rating_vocab.lookup_index(index)

#@ Evaluation of the Model:
review = "I am annoyed with the Movie."
classifier = classifier.to(args.device)
prediction = predict_rating(review, classifier, vectorizer, decision_threshold=0.5)
print("{}: {}".format(review, prediction))

AttributeError: 'ReviewVectorizer' object has no attribute 'vectorize'